In this notebook we'll do further processing of the input text (in the spirit of hyperparameter tuning, rather than cleaning etc).  We'll then build and train a simple RNN classifier.

In [1]:
from __future__ import unicode_literals, print_function

import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, log_loss
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, GRU, Bidirectional, Activation, Dropout
from keras.layers.core import Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

import utils

%load_ext autoreload

Using TensorFlow backend.


### Read in data

In [50]:
%%time

filepath = '../data/data-False-3.pkl'

df, keep_stops, min_sents, vocab, w2i, i2w = utils.read_dataset(filepath)

CPU times: user 14.2 s, sys: 72 ms, total: 14.3 s
Wall time: 14.2 s


### Limit vocab size, pad sequences, and split data

In [51]:
maxlen = 200      # Max number of tokens in input sequence
topn = 10000      # Keep only the top n words in vocabulary.  None means "all"
test_size = 0.1  # Fraction of samples to keep out of training

In [52]:
%%time

# Filter out uncommon words.

df['encoded_text'] = utils.filter_top_words(list(df['encoded_text']), topn)

CPU times: user 876 ms, sys: 52 ms, total: 928 ms
Wall time: 893 ms


In [53]:
%%time

# Pad sequences, using topn as the index for "no token"

X_train = pad_sequences(df['encoded_text'], maxlen=maxlen, value=topn, 
                        padding='post', truncating='post')

Y_train = df['encoded_label'].tolist()

CPU times: user 316 ms, sys: 24 ms, total: 340 ms
Wall time: 327 ms


In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=test_size)

X_train.shape, X_test.shape

((37725, 200), (4192, 200))

In [55]:
# Delete the dataframe, we are done with it for now!
del df

### Define model

This is an important part, so I'll be explicit here rather than hiding things in `utils :)`

In [56]:
embed_dim = 64         # embedding dimension for word vecs
gru_dim = 64           # dimension of GRU layers
bidirectional = False   # whether to use bidirectional
dense_dim = 32         # dimensionality of dense layer
dropout = 0.1           # dropout ratio
batch_size = 64         # batch size
validation_split = 0.15 # Fraction of samples to keep out for validation
max_epochs = 20         # maximum number of epochs to run for

In [57]:
# About 43% of articles are conservative, so let's weight samples accordingly

class_weight = {0: 0.57, 1: 0.43}

In [58]:
# Let's save our best current checkpoints, and stop if we haven't improved in 3 iterations w.r.t. val_acc.

model_dir = 'models'
basename = '{}_{}_{}_{}_{}_{}_{}_{}_{}'.format(gru_dim, embed_dim, dense_dim, dropout, bidirectional,
                                               maxlen, topn, test_size, batch_size) 
filepath = os.path.join(model_dir, basename + '_{epoch:03d}_{val_loss:.5f}_{val_acc:.5f}.h5')

print('saving checkpoints to: {}'.format(filepath))

model_checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, 
                                   save_weights_only=False, mode='auto', period=1)

early_stopping = EarlyStopping(monitor='val_acc', patience=3)

saving checkpoints to: models/64_64_32_0.1_False_200_10000_0.1_64_{epoch:03d}_{val_loss:.5f}_{val_acc:.5f}.h5


We'll store each layer as a variable so we can easily access it later.

In [59]:
model = Sequential()

model.add(Embedding(topn+1, embed_dim, input_length=maxlen))

gru_layer = GRU(gru_dim, dropout=dropout, recurrent_dropout=dropout)

if bidirectional:
  gru_layer = Bidirectional(gru_layer)

model.add(gru_layer)
  
model.add(Dropout(dropout))
model.add(Dense(dense_dim, activation='relu'))

model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [60]:
# Compile the model

_ = model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 64)           640064    
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_10 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total params: 666,945
Trainable params: 666,945
Non-trainable params: 0
_________________________________________________________________


### Train!

In [61]:
hist = model.fit(X_train, Y_train, epochs=max_epochs, batch_size=batch_size,
                 class_weight=class_weight, validation_split=validation_split, 
                 shuffle=True, callbacks=[model_checkpoint, early_stopping])

Train on 32066 samples, validate on 5659 samples
Epoch 1/20
32066/32066 [==============================] - 58s - loss: 0.3072 - acc: 0.6640 - val_loss: 0.2915 - val_acc: 0.6902
Epoch 2/20
32066/32066 [==============================] - 57s - loss: 0.2411 - acc: 0.7669 - val_loss: 0.2291 - val_acc: 0.7881
Epoch 3/20
32066/32066 [==============================] - 57s - loss: 0.1749 - acc: 0.8404 - val_loss: 0.2067 - val_acc: 0.8185
Epoch 4/20
32066/32066 [==============================] - 57s - loss: 0.1345 - acc: 0.8811 - val_loss: 0.2317 - val_acc: 0.8031
Epoch 5/20
32066/32066 [==============================] - 57s - loss: 0.1035 - acc: 0.9136 - val_loss: 0.2676 - val_acc: 0.8017
Epoch 6/20
32066/32066 [==============================] - 57s - loss: 0.0809 - acc: 0.9347 - val_loss: 0.2976 - val_acc: 0.7986
Epoch 7/20
32066/32066 [==============================] - 57s - loss: 0.0630 - acc: 0.9498 - val_loss: 0.3351 - val_acc: 0.8023


### Simple evaluation metrics on hold-out test set

In [62]:
# Load the best checkpoint

model = load_model('models/64_64_32_0.1_False_200_10000_0.1_64_002_0.20667_0.81852.h5')

In [63]:
# Accuracy

scores = model.evaluate(X_test, Y_test, verbose=0)
benchmark = 100. * len(np.where(np.array(Y_test) == 1)[0]) / len(Y_test)

print('Accuracy:  {:2.1f}%'.format(scores[1] * 100.))
print('Benchmark: {:2.1f}% (guessing all label 1)'.format(benchmark))

Accuracy:  81.6%
Benchmark: 56.7% (guessing all label 1)


In [64]:
# F1 score

Y_pred = model.predict_classes(X_test, verbose=0).squeeze().tolist()
benchmark = np.ones_like(Y_test)

print('f1 score:  {:2.2f}'.format(f1_score(Y_test, Y_pred)))
print('Benchmark: {:2.2f} (guessing all label 1): '.format(f1_score(Y_test, benchmark)))

f1 score:  0.84
Benchmark: 0.72 (guessing all label 1): 


In [65]:
# logloss

Y_pred = model.predict(X_test, verbose=0).squeeze().tolist()
benchmark = 0.57 * np.ones_like(Y_test)

print('Log loss:  {:2.2f}'.format(log_loss(Y_test, Y_pred)))
print('Benchmark: {:2.2f} (guessing 0.57 probability "liberal"): '.format(log_loss(Y_test, benchmark)))

Log loss:  0.41
Benchmark: 0.68 (guessing 0.57 probability "liberal"): 


Definitely better than random! `:)`